# 종로구 (1102)
- 작성자 : 황다은
- 작성일자 : 2020년 9월 19일

In [62]:
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

dat = dt.datetime.today().date() # 오늘 날짜

### 1. Raw File 불러오기

In [63]:
df = pd.read_csv("200919one_shot.csv")
soup = BeautifulSoup(df['html'][0], 'html.parser')

### 2. 간단한 전처리 하기

In [64]:
rows = soup.select('div.coronal-popup-table')
table = rows[0].select('table')

In [66]:
df = pd.read_html(str(table[0]))[0]
start = df.iloc[0,0]


### 3. 저장할 DataFrame 미리 만들기

In [74]:
pat_col = ['num', 'dat', 'pth', 'syy', 'syd', 'cnd', 'rsd', 'atn', 'dch', 'adt','age','sex']
rou_col = ['num', 'ord', 'dat', 'rgl', 'rgt', 'frm', 'exd', 'msk', 'ste', 'mob']
patient = pd.DataFrame(columns = pat_col)
route = pd.DataFrame(columns = rou_col)

### 4. 귀찮은 전처리해주기

In [75]:
for i in tqdm(range(int(start)+1)):
    try:
        num = '1102' + '0' * (5- len(str(i))) + str(i)
        sub = df['연 번']== str(i)
        subdb = df[sub]
        pth = subdb.iloc[0,1].split('/')[0]
        cnd = subdb.iloc[0,1].split('/')[1][4:]
        syy = ''; syd = ''; rsd = ''; atn = ''; dch = ''; adt = ''; age = ''; sex = ''
        new_pat = pd.DataFrame([(num, dat, pth, syy, syd, cnd, rsd, atn, dch, adt, age, sex)], columns = pat_col)
        patient = pd.concat([patient, new_pat])

        new_route = subdb.iloc[1:len(subdb)-2,:].copy()
        new_route['mob'] = subdb.iloc[len(subdb)-1,2].split(',')[0]
        new_route['msk'] = subdb.iloc[len(subdb)-1,2].split(',')[1]
        new_route['num'] = num
        new_route['ord'] = range(1, len(new_route)+1)
        new_route['dat'] = dat
        new_route = new_route.drop(['연 번'], axis = 1)
        new_route.columns = ['rgt', 'frm', 'rgl', 'exd', 'ste', 'mob', 'msk', 'num', 'ord', 'dat']
        new_route = new_route.loc[:,rou_col]

        route = pd.concat([route, new_route])
    except:
        continue

100%|█████████████████████████████████████████████████████████████████████████████████| 85/85 [00:00<00:00, 373.81it/s]


In [76]:
patient

,num,dat,pth,syy,syd,cnd,rsd,atn,dch,adt,age,sex
0,110200068,2020-09-19,영등포구 집단발병 일년정종 서울포교소 관련,,,9.7.,,,,,,
0,110200069,2020-09-19,감염경로 추정중,,,9. 8.,,,,,,
0,110200070,2020-09-19,감염경로 추적 중,,,9. 9.,,,,,,
0,110200071,2020-09-19,감염경로 추적 중,,,9. 9.,,,,,,
0,110200072,2020-09-19,감염경로 추적 중,,,9. 9.,,,,,,
0,110200073,2020-09-19,감염경로 추적 중,,,9. 9.,,,,,,
0,110200074,2020-09-19,감염경로 추적 중,,,9. 9.,,,,,,
0,110200075,2020-09-19,종로#73 접촉,,,9. 10.,,,,,,
0,110200076,2020-09-19,감염경로 추적 중,,,9. 9.,,,,,,
0,110200077,2020-09-19,종로#74 접촉,,,9. 10.,,,,,,


In [77]:
route

,num,ord,dat,rgl,rgt,frm,exd,msk,ste,mob
105,110200069,1,2020-09-19,청운효자동,식당,OO식당,2020.9.3.(목) 07:15~07:50 2020.9.7.(월) 10:40~1...,마스크 착용함,소독완료,지하철/도보 이용
106,110200069,2,2020-09-19,청운효자동,식당,OO분식,2020.9.4.(금) 10:30~11:10,마스크 착용함,소독완료,지하철/도보 이용
107,110200069,3,2020-09-19,청운효자동,약국,OO약국,2020.9.6.(일) 13:02~13:05,마스크 착용함,소독완료,지하철/도보 이용
108,110200069,4,2020-09-19,종로17길 36,미용실,낙원이발관,2020.9.4.(금) 17:10~18:00,마스크 착용함,소독완료,지하철/도보 이용
109,110200069,5,2020-09-19,사직동,종합가전,OO대리점,2020.9.7.(월) 15:00~15:56,마스크 착용함,소독완료,지하철/도보 이용
...,...,...,...,...,...,...,...,...,...,...
8,110200083,3,2020-09-19,숭인제2동,"슈퍼, 마트",OO마트,"2020.09.03.(목) 19:12~19:17, 20:33~20:38",마스크 착용함,소독완료,지하철/도보 이용
9,110200083,4,2020-09-19,창신제3동,"병,의원",OO의원,2020.09.07.(월) 11:39~12:30,마스크 착용함,소독완료,지하철/도보 이용
10,110200083,5,2020-09-19,종로1234가동,약국,OO약국,2020.09.07.(월) 12:30~12:34,마스크 착용함,소독완료,지하철/도보 이용
1,110200084,1,2020-09-19,숭인제1동,사무실,OO통상,2020.09.12.(토) 10:10~17:00 2020.09.14.(월) 10:...,마스크 착용함,소독완료,지하철/도보 이용
